## Step 1: Pull Data from yahoo finance

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

import plotly.offline as p
import plotly.graph_objs as go

p.init_notebook_mode(connected=True)
from IPython.display import Image

In [134]:
url_key_statistics = "https://finance.yahoo.com/quote/TSLA/key-statistics"
url_holders = "https://finance.yahoo.com/quote/TSLA/holders"
url_sustainability = "https://finance.yahoo.com/quote/TSLA/sustainability"
url_SnP_Wiki = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [66]:
response = requests.get(url_sustainability)

In [67]:
# response.text

In [68]:
beautsoup_object = BeautifulSoup(response.text, 'html.parser')
# Tells BeautifulSoup that we'd like to use the 'html' parser

In [108]:
# print(beautsoup_object.prettify())

In [70]:
# beautsoup_object.select(".Fz(36px)") -- alternate way but below is more accurate

In [71]:
beautsoup_object.find_all('div', class_ = "Fz(36px)", attrs={"data-reactid": "20"})[0].text

'61'

In [124]:
def getBeatifulResponse(url):
    """Function that gets the response using the URL"""
    response = requests.get(url)
    beautsoup_object = BeautifulSoup(response.text, 'html.parser')
#     print(beautsoup_object.prettify())
    return beautsoup_object

In [172]:
def getSustainabilityScore(url):
    """Function that calls getBeatifulResponse and then extracts 
    Sustainability score from the response"""
    score = None
    try:
        bsoupobj = getBeatifulResponse(url)
        score = int(bsoupobj.find_all('div', class_ = "Fz(36px)", attrs={"data-reactid": "20"})[0].text)
    finally:
        return score
    

In [173]:
getSustainabilityScore(url_sustainability)

In [177]:
def getHolders(url):
    """Function that calls getBeatifulResponse and then extracts 
    list of holders from the response"""
    bsoupobj = getBeatifulResponse(url)
    table = bsoupobj.find_all('td', {'class': 'Py(10px) Va(m) Fw(b) W(15%)'})
    holders = []
    try:
        for row in table:
            holder = row.text
            holders.append(holder)
    finally:
        return holders

In [157]:
getHolders(url_holders)

['25.19%', '59.53%', '79.57%', '1,005']

In [153]:
bsoupobj = getBeatifulResponse("https://finance.yahoo.com/quote/TSLA/holders")
# print(bsoupobj.prettify())
# bsoupobj.find_all(class_ = "Py(10px)")
table = bsoupobj.find_all('td', {'class': 'Py(10px) Va(m) Fw(b) W(15%)'})
holders = []
for row in table:
    print(row)
    holder = row.text
    holders.append(holder)
holders

<td class="Py(10px) Va(m) Fw(b) W(15%)" data-reactid="26">25.19%</td>
<td class="Py(10px) Va(m) Fw(b) W(15%)" data-reactid="30">59.53%</td>
<td class="Py(10px) Va(m) Fw(b) W(15%)" data-reactid="34">79.57%</td>
<td class="Py(10px) Va(m) Fw(b) W(15%)" data-reactid="38">1,005</td>


['25.19%', '59.53%', '79.57%', '1,005']

In [192]:
def getSnPCompanies(url):
    """Function that scrapes the list of S&P 500 companies from Wikipedia"""
    bsoupobj = getBeatifulResponse(url)
    table = bsoupobj.find('table', {'class': 'wikitable sortable'})
    companies = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        company_name = row.findAll('td')[1].text
        companies.append([ticker,company_name])
    return companies

In [198]:
# companies = getSnPCompanies(url_SnP_Wiki)

In [215]:
# companies

In [189]:
bsoupobj = getBeatifulResponse("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
# print(bsoupobj.prettify())
table = bsoupobj.find('table', {'class': 'wikitable sortable'})
tickers = []
company_names = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    company_name = row.findAll('td')[1].text
#     print(ticker)
    tickers.append(ticker)
    company_names.append(company_name)

In [216]:
# company_names

In [205]:
# companies_info = []
# for company in companies[:10]:
#     url_key_statistics = "https://finance.yahoo.com/quote/" + company[0] + "/key-statistics"
#     url_sustainability = "https://finance.yahoo.com/quote/" + company[0] + "/sustainability"
#     url_holders = "https://finance.yahoo.com/quote/"        + company[0] + "/holders"
#     sustainability = getSustainabilityScore(url_sustainability)
#     holders = getHolders(url_holders)

#     company_stats ={}
#     company_stats['COMPANY'] = company[0]
#     company_stats['SUSTAINABILITY_SCORE'] = sustainability
#     company_stats['HOLDERS'] = holders
    
#     companies_info.append(company_stats)
# #     print (company_stats)
# #     print(companies)
#     if len(companies_info) == 1:
#         print('Scraping data', end="")
#     print('.', end="")
# companies_info

## Step 2: Process data into data structures

In [217]:
companies = getSnPCompanies(url_SnP_Wiki)

In [218]:
companies_info = []
for company in companies[:10]:
    url_key_statistics = "https://finance.yahoo.com/quote/" + company[0] + "/key-statistics"
    url_sustainability = "https://finance.yahoo.com/quote/" + company[0] + "/sustainability"
    url_holders = "https://finance.yahoo.com/quote/"        + company[0] + "/holders"
    sustainability = getSustainabilityScore(url_sustainability)
    holders = getHolders(url_holders)

    company_stats ={}
    company_stats['TICKER'] = company[0]
    company_stats['COMPANY_NAME'] = company[1]
    company_stats['SUSTAINABILITY_SCORE'] = sustainability
    company_stats['INSIDER_HOLDERS'] = holders[0]
    company_stats['INSTITUTION_HOLDERS'] = holders[1]
    company_stats['INSTITUTION_FLOAT_HOLDERS'] = holders[2]
    company_stats['INSTITUTION_HOLDERS_NO'] = int(holders[3].replace(',',''))
    
    companies_info.append(company_stats)
#     print (company_stats)
#     print(companies)
    if len(companies_info) == 1:
        print('Scraping data', end="")
    print('.', end="")
companies_info

Scraping data..........

[{'TICKER': 'MMM',
  'COMPANY_NAME': '3M Company',
  'SUSTAINABILITY_SCORE': 66,
  'INSIDER_HOLDERS': '7.74%',
  'INSTITUTION_HOLDERS': '10.42%',
  'INSTITUTION_FLOAT_HOLDERS': '11.29%',
  'INSTITUTION_HOLDERS_NO': 2},
 {'TICKER': 'ABT',
  'COMPANY_NAME': 'Abbott Laboratories',
  'SUSTAINABILITY_SCORE': 64,
  'INSIDER_HOLDERS': '0.72%',
  'INSTITUTION_HOLDERS': '74.53%',
  'INSTITUTION_FLOAT_HOLDERS': '75.07%',
  'INSTITUTION_HOLDERS_NO': 2194},
 {'TICKER': 'ABBV',
  'COMPANY_NAME': 'AbbVie Inc.',
  'SUSTAINABILITY_SCORE': 66,
  'INSIDER_HOLDERS': '0.09%',
  'INSTITUTION_HOLDERS': '71.54%',
  'INSTITUTION_FLOAT_HOLDERS': '71.60%',
  'INSTITUTION_HOLDERS_NO': 2458},
 {'TICKER': 'ABMD',
  'COMPANY_NAME': 'ABIOMED Inc',
  'SUSTAINABILITY_SCORE': None,
  'INSIDER_HOLDERS': '4.22%',
  'INSTITUTION_HOLDERS': '87.21%',
  'INSTITUTION_FLOAT_HOLDERS': '91.05%',
  'INSTITUTION_HOLDERS_NO': 688},
 {'TICKER': 'ACN',
  'COMPANY_NAME': 'Accenture plc',
  'SUSTAINABILITY_SCORE': 75,
  'INSIDER_HOLDER

## Step 3: Visualize
